# Query Data using LLM

Here is the overall RAG pipeline.   In this notebook, we will do steps (5), (6), (7), (8), (9)
- Importing data is already done in this notebook [rag_1_B_load_data.ipynb](rag_1_B_load_data.ipynb)
- 👉 Step 5: Calculate embedding for user query
- 👉 Step 6 & 7: Send the query to vector db to retrieve relevant documents
- 👉 Step 8 & 9: Send the query and relevant documents (returned above step) to LLM and get answers to our query

![image missing](../media/rag-overview-2.png)

## Configuration

In [1]:
class MyConfig:
    pass
MY_CONFIG = MyConfig()

MY_CONFIG.EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
MY_CONFIG.EMBEDDING_LENGTH = 384

MY_CONFIG.DB_URI = './rag_1_dpk.db'  # For embedded instance
#MY_CONFIG.DB_URI = 'http://localhost:19530'  # For Docker instance
MY_CONFIG.COLLECTION_NAME = 'dpk_walmart_docs'

MY_CONFIG.LLM_MODEL = "meta/meta-llama-3-8b-instruct"


## Configuration

Create a .env file with the following properties.  You can use [env.txt](../env.txt) as starting point

---

```text
REPLICATE_API_TOKEN=YOUR_TOKEN_GOES_HERE
```

---

## Load Configurations


In [2]:
import os,sys
## Load Settings from .env file
from dotenv import find_dotenv, dotenv_values

# _ = load_dotenv(find_dotenv()) # read local .env file
config = dotenv_values(find_dotenv())

# debug
# print (config)

MY_CONFIG.REPLICATE_API_TOKEN = config.get('REPLICATE_API_TOKEN')

if  MY_CONFIG.REPLICATE_API_TOKEN:
    print ("✅ config REPLICATE_API_TOKEN found")
else:
    raise Exception ("'❌ REPLICATE_API_TOKEN' is not set.  Please set it above to continue...")


✅ config REPLICATE_API_TOKEN found


## Connect to Vector Database

Milvus can be embedded and easy to use.

<span style="color:blue;">Note: If you encounter an error about unable to load database, try this: </span>

- <span style="color:blue;">In **vscode** : **restart the kernel** of previous notebook. This will release the db.lock </span>
- <span style="color:blue;">In **Jupyter**: Do `File --> Close and Shutdown Notebook` of previous notebook. This will release the db.lock</span>
- <span style="color:blue;">Re-run this cell again</span>


In [3]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(MY_CONFIG.DB_URI)

print ("✅ Connected to Milvus instance:", MY_CONFIG.DB_URI)

✅ Connected to Milvus instance: ./rag_1_dpk.db


## Step-: Setup Embeddings

Use the same embeddings we used to index our documents!

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(MY_CONFIG.EMBEDDING_MODEL)

def get_embeddings (str):
    embeddings = model.encode(str, normalize_embeddings=True)
    return embeddings

/home/sujee/apps/anaconda3/envs/data-prep-kit-2/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/sujee/apps/anaconda3/envs/data-prep-kit-2/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# Test embeddings
embeddings = get_embeddings('Paris 2024 Olympics')
print ('embeddings len =', len(embeddings))
print ('embeddings[:5] = ', embeddings[:5])

embeddings len = 384
embeddings[:5] =  [ 0.02468893  0.10352128  0.02752643 -0.08551716 -0.01412826]


## Vector Search and RAG

In [6]:
# Get relevant documents using vector / sementic search

def fetch_relevant_documents (query : str) :
    search_res = milvus_client.search(
        collection_name=MY_CONFIG.COLLECTION_NAME,
        data = [get_embeddings(query)], # Use the `emb_text` function to convert the question to an embedding vector
        limit=3,  # Return top 3 results
        search_params={"metric_type": "IP", "params": {}},  # Inner product distance
        output_fields=["text"],  # Return the text field
    )
    # print (search_res)

    retrieved_docs_with_distances = [
        {'text': res["entity"]["text"], 'distance' : res["distance"]} for res in search_res[0]
    ]
    return retrieved_docs_with_distances
## --- end ---


In [7]:
# test relevant vector search
import json
import pprint

question = "What was Walmart's revenue in 2023?"
relevant_docs = fetch_relevant_documents(question)
pprint.pprint(relevant_docs, indent=4)

[   {   'distance': 0.5978392958641052,
        'text': 'Stock Performance Chart\n'
                'Walmart Inc., 2019 = $100.00. Walmart Inc., 2020 = $120.27. '
                'Walmart Inc., 2021 = $148.41. Walmart Inc., 2022 = $148.47. '
                'Walmart Inc., 2023 = $153.58. Walmart Inc., 2024 = $177.30. '
                'S&P 500 Index, 2019 = 100.00. S&P 500 Index, 2020 = 121.68. '
                'S&P 500 Index, 2021 = 142.67. S&P 500 Index, 2022 = 175.90. '
                'S&P 500 Index, 2023 = 161.45. S&P 500 Index, 2024 = 195.06. '
                'S&P 500 Consumer   Discretionary, 2019 = . S&P 500 Consumer   '
                'Discretionary, 2020 = . S&P 500 Consumer   Discretionary, '
                '2021 = . S&P 500 Consumer   Discretionary, 2022 = . S&P 500 '
                'Consumer   Discretionary, 2023 = . S&P 500 Consumer   '
                'Discretionary, 2024 = . Discretionary   Distribution &   '
                'RiliId, 2019 = 100.00. Discretionary   

## Initialize LLM

### LLM Choices at Replicate

- llama 3.1 : Latest
    - **meta/meta-llama-3.1-405b-instruct** : Meta's flagship 405 billion parameter language model, fine-tuned for chat completions
- Base version of llama-3 from meta
    - [meta/meta-llama-3-8b](https://replicate.com/meta/meta-llama-3-8b) : Base version of Llama 3, an 8 billion parameter language model from Meta.
    - **meta/meta-llama-3-70b** : 70 billion
- Instruct versions of llama-3 from meta, fine tuned for chat completions
    - **meta/meta-llama-3-8b-instruct** : An 8 billion parameter language model from Meta, 
    - **meta/meta-llama-3-70b-instruct** : 70 billion

References 

- https://docs.llamaindex.ai/en/stable/examples/llm/llama_2/?h=replicate

In [8]:
import os
os.environ["REPLICATE_API_TOKEN"] = MY_CONFIG.REPLICATE_API_TOKEN

In [9]:
import replicate

def ask_LLM (question, relevant_docs):
    context = "\n".join(
        [doc['text'] for doc in relevant_docs]
    )
    print ('============ context (this is the context supplied to LLM) ============')
    print (context)
    print ('============ end  context ============', flush=True)

    system_prompt = """
    Human: You are an AI assistant. You are able to find answers to the questions from the contextual passage snippets provided.
    """
    user_prompt = f"""
    Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
    <context>
    {context}
    </context>
    <question>
    {question}
    </question>
    """

    print ('============ here is the answer from LLM... STREAMING... =====')
    # The meta/meta-llama-3-8b-instruct model can stream output as it's running.
    for event in replicate.stream(
        MY_CONFIG.LLM_MODEL,
        input={
            "top_k": 0,
            "top_p": 0.95,
            "prompt": user_prompt,
            "max_tokens": 512,
            "temperature": 0.1,
            "system_prompt": system_prompt,
            "length_penalty": 1,
            "max_new_tokens": 512,
            "stop_sequences": "<|end_of_text|>,<|eot_id|>",
            "prompt_template": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
            "presence_penalty": 0,
            "log_performance_metrics": False
        },
    ):
        print(str(event), end="")
    ## ---
    print ('\n======  end LLM answer ======\n', flush=True)


In [10]:
import replicate

def ask_LLM (question, relevant_docs):
    context = "\n".join(
        [doc['text'] for doc in relevant_docs]
    )
    print ('============ context (this is the context supplied to LLM) ============')
    print (context)
    print ('============ end  context ============', flush=True)

    system_prompt = """
    Human: You are an AI assistant. You are able to find answers to the questions from the contextual passage snippets provided.
    """
    user_prompt = f"""
    Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
    <context>
    {context}
    </context>
    <question>
    {question}
    </question>
    """

    print ('============ here is the answer from LLM... STREAMING... =====')
    # The meta/meta-llama-3-8b-instruct model can stream output as it's running.
    for event in replicate.stream(
        MY_CONFIG.LLM_MODEL,
        input={
            "top_k": 0,
            "top_p": 0.95,
            "prompt": user_prompt,
            "max_tokens": 512,
            "temperature": 0.1,
            "system_prompt": system_prompt,
            "length_penalty": 1,
            "max_new_tokens": 512,
            "stop_sequences": "<|end_of_text|>,<|eot_id|>",
            "prompt_template": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
            "presence_penalty": 0,
            "log_performance_metrics": False
        },
    ):
        print(str(event), end="")
    ## ---
    print ('\n======  end LLM answer ======\n', flush=True)


## Query

In [11]:
%%time

question = "What was Walmart's revenue in 2023?"
relevant_docs = fetch_relevant_documents(question)
ask_LLM(question=question, relevant_docs=relevant_docs)

============ context (this is the context supplied to LLM) ============
Stock Performance Chart
Walmart Inc., 2019 = $100.00. Walmart Inc., 2020 = $120.27. Walmart Inc., 2021 = $148.41. Walmart Inc., 2022 = $148.47. Walmart Inc., 2023 = $153.58. Walmart Inc., 2024 = $177.30. S&P 500 Index, 2019 = 100.00. S&P 500 Index, 2020 = 121.68. S&P 500 Index, 2021 = 142.67. S&P 500 Index, 2022 = 175.90. S&P 500 Index, 2023 = 161.45. S&P 500 Index, 2024 = 195.06. S&P 500 Consumer   Discretionary, 2019 = . S&P 500 Consumer   Discretionary, 2020 = . S&P 500 Consumer   Discretionary, 2021 = . S&P 500 Consumer   Discretionary, 2022 = . S&P 500 Consumer   Discretionary, 2023 = . S&P 500 Consumer   Discretionary, 2024 = . Discretionary   Distribution &   RiliId, 2019 = 100.00. Discretionary   Distribution &   RiliId, 2020 = 117.54. Discretionary   Distribution &   RiliId, 2021 = 166.19. Discretionary   Distribution &   RiliId, 2022 = 180.56. Discretionary   Distribution &   RiliId, 2023 = 147.66. Discre

In [12]:
%%time

question = "How many distribution centers does Walmart have?"
relevant_docs = fetch_relevant_documents(question)
ask_LLM(question=question, relevant_docs=relevant_docs)

============ context (this is the context supplied to LLM) ============
"At Walmart, we're a people-led, tech-powered omnichannel retailer dedicated
through, up to and including 2030. Additional qualifying information can be found by visiting http://corporate.walmart.com/purpose/esgreport.
Stock Performance Chart
Walmart Inc., 2019 = $100.00. Walmart Inc., 2020 = $120.27. Walmart Inc., 2021 = $148.41. Walmart Inc., 2022 = $148.47. Walmart Inc., 2023 = $153.58. Walmart Inc., 2024 = $177.30. S&P 500 Index, 2019 = 100.00. S&P 500 Index, 2020 = 121.68. S&P 500 Index, 2021 = 142.67. S&P 500 Index, 2022 = 175.90. S&P 500 Index, 2023 = 161.45. S&P 500 Index, 2024 = 195.06. S&P 500 Consumer   Discretionary, 2019 = . S&P 500 Consumer   Discretionary, 2020 = . S&P 500 Consumer   Discretionary, 2021 = . S&P 500 Consumer   Discretionary, 2022 = . S&P 500 Consumer   Discretionary, 2023 = . S&P 500 Consumer   Discretionary, 2024 = . Discretionary   Distribution &   RiliId, 2019 = 100.00. Discretiona

In [13]:
%%time

question = "When was the moon landing?"
relevant_docs = fetch_relevant_documents(question)
ask_LLM(question=question, relevant_docs=relevant_docs)

============ context (this is the context supplied to LLM) ============
                                                 -                                                                        
3/29/2024 10:28:40 AM
*E@4< '6C7@C>2?46  92CE
 &$' ) *&% &    0  )  ,$,# + -  +&+ # ) +,)% 
 &7-  59.:&0*287 &2) %36/.2,  &4.8&0  *+.(.8 
:D42= 062CD  ?565 !2?F2CJ   
============ end  context ============
============ here is the answer from LLM... STREAMING... =====
I'm happy to help! However, I must point out that the provided context does not contain any information about the moon landing. The text appears to be a jumbled mix of characters and symbols, and does not provide any relevant information about the moon landing or any other historical event.

If you could provide a different context or question, I would be happy to try and assist you to the best of my abilities.
======  end LLM answer ======

CPU times: user 268 ms, sys: 12.4 ms, total: 280 ms
Wall time: 1.37 s
